THIS IS A CODE TO CREATE CSV FOR MEDIS ATTACK OUTPUT IN TARGETED ATTACK SETTINGS FOR 10 STEPS.

[POLYP SEGMENTATION TASK]


In [ ]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive')

# SET DIRECTORY
import os,random
os.chdir('/content/drive/MyDrive/PraNet/')

# INSTALL PACKAGES
!pip install pytorch-msssim
!pip install kornia

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.4/653.4 kB 29.3 MB/s eta 0:00:00


In [ ]:
# IMPORT ALL REQUIRED LIBRARIES
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os, argparse
from scipy import misc
from lib.PraNet_Res2Net import PraNet
from utils.dataloader import test_dataset,transforms
import imageio
import io
import cv2
import torch.nn.functional as nnf
from skimage import img_as_ubyte
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from sklearn.metrics import confusion_matrix
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import kornia
from LovaszSoftmax.pytorch import lovasz_losses as L
import csv
from csv import DictReader
import warnings
warnings.filterwarnings('ignore')

# SET PATH AND VARIABLES
use_cuda = True
pretrained_model = 'snapshots/PraNet_Res2Net/PraNet-19.pth'
test_dataset_path = 'TestDataset/'
save_result_path =  '/content/drive/MyDrive/PraNet/code_check_medis/attack_10steps_results/'

# SET DEVICE
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  False


In [ ]:
%%capture
# LOAD MODEL
model = PraNet().to(device) # Define Model
model.load_state_dict(torch.load(pretrained_model )) # Load pretrained model
model.eval() # Set the model in evaluation mode

In [ ]:
# ALL REQUIRED FUNCTIONS

#_______________________GET TARGET__________________________#

def get_target(targetPath,test_loader,targetType,shape):
  resize = transforms.Resize((500,540))
  if targetType is 'black':
    target = torch.zeros(shape).to(device)
  elif targetType is 'white':
    target = torch.ones(shape).to(device)
  else:
    target = test_loader.binary_loader(path = targetPath)
    if 'Kvasir' in targetPath:
      target = resize(target)
    target = np.asarray(target, np.float32)
    target /= (target.max() + 1e-8)
    target = torch.from_numpy(target).to(device)
  return target

#_______________________GET PREDICTION__________________________#

def get_prediction(result,target):
  res = result
  res = F.upsample(res, size=target.shape, mode='bilinear', align_corners=False)
  res = res.sigmoid().squeeze()
  res = (res - res.min()) / (res.max() - res.min() + 1e-8)
  return res

#_______________________CALCULATE METRICS__________________________#

def calculate_metrics(prediction,target,metricName):

  #_1.___________mean Iou___________#
  if metricName is 'MeanIoU':
    y_pred = prediction.flatten()
    y_true = target.flatten()
    current = confusion_matrix(y_true.round(), y_pred.round(), labels=[0, 1])
    intersection = np.diag(current)
    ground_truth_set = current.sum(axis=1)
    predicted_set = current.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / union.astype(np.float32)
    MeanIoU = np.nanmean(IoU)
    return round(MeanIoU,3)

  #_2.___________ dice___________#
  elif metricName is 'Dice':
    intersection = 2.0 * (prediction * target).sum()
    union = prediction.sum() + target.sum()
    if prediction.sum() == 0 and target.sum() == 0:
      Dice = 1.0
    else:
      Dice = intersection / union
    return Dice


#_______________________CALCULATE LOSS__________________________#

def calculate_loss(prediction,target,Loss):
  bce = nn.BCELoss()
  if Loss is 'wBCE_wIoU_Loss': #__________weighted bce + weighted iou___________
    z = (1,1)
    target = torch.reshape(target,z + target.shape)
    prediction = torch.reshape(prediction,z + prediction.shape)
    weit = 1 +  5*torch.abs(F.avg_pool2d(target, kernel_size=31, stride=1, padding=15) - target)
    wbce = F.binary_cross_entropy_with_logits(prediction, target, reduce='none')
    wbce = (weit*wbce).sum(dim=(2, 3)) / weit.sum(dim=(2, 3))
    prediction = torch.sigmoid(prediction)
    inter = ((prediction * target)*weit).sum(dim=(2, 3))
    union = ((prediction + target)*weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1)/(union - inter+1)
    loss = (wbce + wiou).mean()

  elif Loss is 'Lovasz_Hinge_Loss': #__________lovasz_hinge___________
    loss = L.lovasz_hinge(prediction, target, per_image = True,ignore=None)

  elif Loss is 'Log_Cosh_Dice_Loss': #__________logcoshdice___________
    loss = torch.log(torch.cosh(1-calculate_metrics(prediction,target,'Dice')))


  elif Loss is 'BCE_Dice_Loss': #__________bce_dice___________
    loss = bce(prediction,target) + (1-calculate_metrics(prediction,target,'Dice'))

  elif Loss is 'Exponential_Log_Loss': #__________exponential_log___________ ########### CHECK FIRST
    gamma = 2.0
    a = 0.5
    L_dice = torch.mean((-torch.log(1-calculate_metrics(prediction,target,'Dice')))**gamma)
    L_bce = torch.mean((-torch.log(bce(prediction,target)))**gamma)
    loss = (a) *  L_dice + (1-a) * L_bce
  return loss

#_______________________SAVE METRICS__________________________#

def save_metrics(test_data,metrics,final_m1,final_m2,targetNum,lossName):
  os.makedirs(save_result_path + '/polyp/' +test_data+'/', exist_ok = True) # CREATE FOLDER FOR SAVING
  os.makedirs(save_result_path + '/polyp/' +test_data+'/'+ lossName + '/', exist_ok = True) #  CREATE FOLDER FOR SAVING
  fileName1 = save_result_path + '/polyp/' +test_data+'/'+lossName+'/target'+str(targetNum+1)+'_predTarget.csv'  # FOR TARGET
  fileName2 = save_result_path + '/polyp/' + test_data+'/'+lossName+'/target'+str(targetNum+1)+'_predGT.csv'       # FOR UNTARGETED
  Header1 = ['Input Image','Target Mask']
  Header2 = ['Input', 'GT']
  for i in range(10):
    Header1.append('Step')
    Header1.append(lossName)
    Header1.extend(metrics)
    Header2.append('Step')
    Header2.append(lossName)
    Header2.extend(metrics)
  with open(fileName1, 'w') as f:
    write = csv.writer(f) # using csv.writer method from CSV package
    write.writerow(Header1)
    write.writerows(final_m1)
  with open(fileName2, 'w') as f:
    write = csv.writer(f) # using csv.writer method from CSV package
    write.writerow(Header2)
    write.writerows(final_m2)

In [ ]:
# BASIC OPERATION

def polyp_attack(test_data,targetPath,targetType,targetNum,metrics,lossName,save_result):
  data_path = test_dataset_path + '{}'.format(test_data)
  input_adv = None
  final_m1 = []
  final_m2 = []
  image_root = '{}/images/'.format(data_path)
  gt_root = '{}/masks/'.format(data_path)
  test_loader = test_dataset(image_root, gt_root, testsize = 352)
  target_dict = targetPath
  print('\nImage count:')

  # start process
  for i in tqdm(range(test_loader.size)): # loop over all images
    img, gt, name = test_loader.load_data()
    img = img.to(device)
    img_unaltered = img.clone().detach().requires_grad_(True) # non-adv image

    # Ground Truth
    resize = transforms.Resize((500,540))
    if test_data == 'Kvasir':
      gt = resize(gt)
    gt = np.asarray(gt, np.float32)
    gt /= (gt.max() + 1e-8)
    gt = torch.from_numpy(gt).to(device)

    # get target
    if targetNum == 1: # extract dict for p2 and p3 target
      p = [v for k,v in target_dict.items() if k == name]
      targetPath = gt_root + p[0]
    target = get_target(targetPath=targetPath,test_loader=test_loader,targetType=targetType,shape = gt.shape)
    if torch.equal(target,gt): # continue process if target mask is same as ground truth mask
      continue
    if not (targetPath == None): # set target name (for saving purpose)
      targetName = targetPath.split("/")[-1]
    else:
      targetName = targetType + '.png'

    # Adversarial Attack
    m = []
    m2 = []
    m.append(name) # for csv (saving purpose)
    m.append(targetName) # for csv (saving purpose)
    m2.append(name) # for csv (saving purpose)
    m2.append(name) # for csv (saving purpose)
    step = 0
    mean_IoU = 0
    while step < 10 and mean_IoU < 0.95:  # loop for IGSM target attack
      m.append(step+1) # for csv (saving purpose)
      m2.append(step+1) # for csv (saving purpose)
      img.requires_grad_(True)  # set grad
      x1 = kornia.geometry.resize(img,((352,352))) # resize image
      x2 = kornia.augmentation.Normalize(torch.tensor([0.485,0.456,0.406]),torch.tensor([0.229,0.224,0.225]))(x1) # normalization
      # get prediction
      res5, res4, res3, res2 = model(x2)
      prediction1 = get_prediction(res2,gt)
      prediction2 = get_prediction(res3,gt)
      prediction3 = get_prediction(res4,gt)
      prediction4 = get_prediction(res5,gt)
      prediction = (prediction1 + prediction2 + prediction3 + prediction4) / 4.0   # total Prediction
      mean_IoU = calculate_metrics(prediction.detach().cpu().numpy(),target.detach().cpu().numpy(),metricName = 'MeanIoU') # calculate mIoU
      # get loss
      if lossName is 'Combined_Loss':
        loss1 = calculate_loss(prediction1,target,'wBCE_wIoU_Loss')
        loss2 = calculate_loss(prediction2,target,'wBCE_wIoU_Loss')
        loss3 = calculate_loss(prediction3,target,'wBCE_wIoU_Loss')
        loss4 = calculate_loss(prediction4,target,'wBCE_wIoU_Loss')
        lossA = loss1 + loss2 + loss3 + loss4

        lossB = calculate_loss(prediction,target,'Log_Cosh_Dice_Loss')
        lossC = calculate_loss(prediction,target,'BCE_Dice_Loss')
        lossD = calculate_loss(prediction,target,'Exponential_Log_Loss')
        lossE = calculate_loss(prediction,target,'Lovasz_Hinge_Loss')

        loss = 0.21 * lossA + 0.18 * lossB + 0.20 * lossC + 0.15 * lossD + 0.26 * lossE # combined loss
      elif lossName is 'wBCE_wIoU_Loss':
        loss1 = calculate_loss(prediction1,target,lossName)
        loss2 = calculate_loss(prediction2,target,lossName)
        loss3 = calculate_loss(prediction3,target,lossName)
        loss4 = calculate_loss(prediction4,target,lossName)
        loss = loss1 + loss2 + loss3 + loss4
      else:
        loss = calculate_loss(prediction,target,lossName)
      model.zero_grad() # make grad zero
      loss.backward() # calculate gradients
      perturbation = (0.003922 * torch.sign(img.grad)) #  calculate perturbation
      input_adv = img.data # saving last input
      img.data = (img.data - perturbation).clamp(0.0,1.0) # apply perturbation into image
      img.grad.data.zero_() # make x_grad zero
      # get metrics
      m.append(round(loss.item(),3))
      m2.append(round(loss.item(),3))
      for metricName in metrics: # loop for 4 metrics
        measure = calculate_metrics(prediction.detach().cpu().numpy(),target.detach().cpu().numpy(),metricName) # pred with target
        measure2 = calculate_metrics(prediction.detach().cpu().numpy(),gt.detach().cpu().numpy(),metricName)   # pred with gt
        m.append(measure)
        m2.append(measure2)
      step += 1
    final_m1.append(m)
    final_m2.append(m2)
  if save_result :
    save_metrics(test_data,metrics,final_m1,final_m2,targetNum,lossName) # save metrics in csv

In [ ]:
# starts from here..{Modify 'Dataloader.py' file first..then run this cell}

data = ['CVC-300','CVC-ClinicDB','CVC-ColonDB','Kvasir'] # testdataset
save_result = True
metrics = ['MeanIoU']
lossNames = ['Log_Cosh_Dice_Loss','BCE_Dice_Loss', 'Exponential_Log_Loss', 'wBCE_wIoU_Loss', 'Lovasz_Hinge_Loss','Combined_Loss']

print("\nLoss count:")
for lossName in tqdm(lossNames): # LOOP FOR EACH LOSS FUNCTION
  print("\nTest Data count:")
  for test_data in tqdm(data):  # loop over all data
    # SET TARGET 1 (RANDOM SELECTION FROM DATA)
    p = test_dataset_path + test_data+'/masks/'
    random_target = random.choice(os.listdir(p))
    p1 = p  + str(random_target)  # randomly selected target (p1)

    # SET TARGET 2 (HIGHEST MIOU OR DICE BASED)
    p2 = {}
    file_name = '/content/drive/MyDrive/PraNet/code_check_medis/' + str(test_data) + '_target2.csv'  # csv file
    with open(file_name, 'r') as read_obj:
      csv_dict_reader = DictReader(read_obj)
      for row in csv_dict_reader:
        d = sorted(row.items(), key=lambda x: x[1], reverse=True)
        p2.update({d[0][1]:d[1][0]}) # {base image: max dice image} # target based on highest miou (p2)

    targetPath = [p1,p2,None,None]  # a list with all target path ____p1:str, p2:dict, None for black and white target
    targetType = [None,None,'white','black']

    print("\nTarget count:")
    for t in tqdm(range(len(targetPath))):# loop over 4 targets
      polyp_attack(test_data,targetPath[t],targetType[t],t,metrics,lossName,save_result) # call main function



Loss count:


  0%|          | 0/6 [00:00<?, ?it/s]


Test Data count:


  0%|          | 0/4 [00:00<?, ?it/s]


Target count:


  0%|          | 0/4 [00:00<?, ?it/s]

60

Image count:


  0%|          | 0/60 [00:00<?, ?it/s]

NameError: ignored